In [2]:
import csv
import json
import os
import re
from datetime import date

from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from tqdm import tqdm

In [3]:
# Configurações do Chrome
header = 'user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36'
chrome_options = Options()
chrome_options.add_argument(header)
chrome_options.add_argument('--headless')

# URL Base
base_url = 'https://www.zapimoveis.com.br/venda/apartamentos/sp+santos/?transacao=venda&onde=%2CPara%C3%ADba%2CCampina+Grande%2C%2C%2C%2C%2Ccity%2CBR%3EParaiba%3ENULL%3ECampina+Grande%2C-6.724744%2C-36.05617%2C&&tipos=apartamento_residencial%2Ccasa_residencial%2Ccondominio_residencial&pagina='


# Página inicial
pagina = 1

# Lista para armazenar os links
todos_links = set()
links_novos = list()

# Carrega os links existentes do CSV
if os.path.exists('artifacts/links_imovel.csv'):
    with open('artifacts/links_imovel.csv', 'r') as arquivo_existente:
        reader = csv.reader(arquivo_existente)
        for linha in reader:
            if linha:
                todos_links.add(linha[0])

# Scrapper
while True:
    url = base_url + str(pagina)
    print(f'Acessando a página {url}...')

    driver = webdriver.Chrome(options=chrome_options)
    driver.get(url=url)

    cards = driver.find_elements(By.CSS_SELECTOR, '[data-cy="rp-property-cd"]')

    try:
        ActionChains(driver).move_to_element(cards[-1]).pause(
            4
        ).move_to_element(cards[-1]).perform()
        WebDriverWait(driver, 10).until(
            EC.visibility_of_all_elements_located((
                By.CSS_SELECTOR,
                '[data-cy="rp-property-cd"]',
            ))
        )
    except:
        print('Nenhum imóvel encontrado ou fim das páginas.')
        driver.close()
        break

    imoveis = driver.find_elements(
        By.CSS_SELECTOR, '[data-cy="rp-property-cd"]'
    )

    links_encontrados = 0
    for imovel in imoveis:
        link = imovel.find_element(By.TAG_NAME, 'a').get_attribute('href')
        if link and link not in todos_links:
            with open('artifacts/links_imovel.csv', 'a', newline='') as link_imoveis:
                writer = csv.writer(link_imoveis)
                writer.writerow([link])
            todos_links.add(link)
            links_novos.append(link)
            links_encontrados += 1

    print(f'Encontrados {links_encontrados} links na página {pagina}.')

    if links_encontrados == 0:
        print('Fim das páginas.')
        print(f'Total de links coletados: {len(todos_links)}')
        driver.close()
        break  # <-- Adicione o break aqui

    driver.close()
    pagina += 1

# Exibe os links coletados
print(f'Foram encontrados: {len(links_novos)} imoveis novos.')
print(f'Total de links: {len(todos_links)}')

Acessando a página https://www.zapimoveis.com.br/venda/apartamentos/sp+santos/?transacao=venda&onde=%2CPara%C3%ADba%2CCampina+Grande%2C%2C%2C%2C%2Ccity%2CBR%3EParaiba%3ENULL%3ECampina+Grande%2C-6.724744%2C-36.05617%2C&&tipos=apartamento_residencial%2Ccasa_residencial%2Ccondominio_residencial&pagina=1...
Encontrados 30 links na página 1.
Acessando a página https://www.zapimoveis.com.br/venda/apartamentos/sp+santos/?transacao=venda&onde=%2CPara%C3%ADba%2CCampina+Grande%2C%2C%2C%2C%2Ccity%2CBR%3EParaiba%3ENULL%3ECampina+Grande%2C-6.724744%2C-36.05617%2C&&tipos=apartamento_residencial%2Ccasa_residencial%2Ccondominio_residencial&pagina=2...
Encontrados 30 links na página 1.
Acessando a página https://www.zapimoveis.com.br/venda/apartamentos/sp+santos/?transacao=venda&onde=%2CPara%C3%ADba%2CCampina+Grande%2C%2C%2C%2C%2Ccity%2CBR%3EParaiba%3ENULL%3ECampina+Grande%2C-6.724744%2C-36.05617%2C&&tipos=apartamento_residencial%2Ccasa_residencial%2Ccondominio_residencial&pagina=2...
Encontrados 25 li

In [4]:
# Adiciona o dicionário de cada imóvel à lista
imoveis = set()

if os.path.exists('imoveis.json'):
    with open('imoveis.json', 'r') as arquivo_existente:
        for linha in arquivo_existente:
            if linha.strip():  # ignora linhas em branco
                dado = json.loads(linha)
                imoveis.add(dado['codigo_zapimoveis'])  # adiciona o ID único

# Filtra as URLs que não contêm '/lancamento/'
todos_links_filtrados = [
   link for link in todos_links if '/lancamento/' not in link
]

# Exibe as URLs filtradas
#print(f'Total de links após filtro: {len(todos_links_filtrados)}')
#todos_links_filtrados

for i, link in enumerate(todos_links_filtrados, start=1):
    print(f'Acessando o link {link}...')
    print(f'Acessando link {i} de {len(todos_links_filtrados)}')

    driver = webdriver.Chrome(options=chrome_options)
    driver.get(url=link)

    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((
                By.CSS_SELECTOR,
                '[data-testid="price-info-value"]',
            ))
        )
        print('Página acessada com sucesso.')
    except:
        print(
            f'Erro 404 encontrado na página {url}. Pulando para a próxima...'
        )
        driver.close()
        # Se o elemento de erro 404 não for encontrado, continua normalmente
        continue

    imovel_info = {}

    # Extraindo o ID do imóvel
    id_numero = re.search(r'id-(\d+)', link)
    if id_numero:
        imovel_info['codigo_zapimoveis'] = id_numero.group(1)
    else:
        imovel_info['codigo_zapimoveis'] = None  # ou outra ação de fallback

    # Extraindo a data de criação
    criado_em_texto = driver.find_element(
        By.CSS_SELECTOR, value='[data-testid="listing-created-date"]'
    ).text

    criado_em = re.search(
        r'criado em (\d{1,2}) de (\w+) de (\d{4})', criado_em_texto
    )

    if criado_em:
        dia, mes_str, ano = criado_em.groups()
        meses = {
            'janeiro': '01',
            'fevereiro': '02',
            'março': '03',
            'abril': '04',
            'maio': '05',
            'junho': '06',
            'julho': '07',
            'agosto': '08',
            'setembro': '09',
            'outubro': '10',
            'novembro': '11',
            'dezembro': '12',
        }
        mes_num = meses[mes_str.lower()]
        imovel_info['data_criacao'] = f'{int(dia):02d}/{mes_num}/{ano}'
    else:
        imovel_info['data_criacao'] = None  # ou outra ação de fallback

    # Extraindo o código do anunciante
    codigo_anunciante = driver.find_element(
        By.CSS_SELECTOR, value='[data-cy="ldp-propertyCodes-txt"]'
    ).text
    imovel_info['codigo_anunciante'] = (
        re.search(r':\s(.+)\s\|', codigo_anunciante).group()[1:-1].strip()
    )

    imovel_info['imobiliaria'] = driver.find_elements(
        By.CSS_SELECTOR, value='[data-testid="official-store-redirect-link"]'
    )[1].text
    imovel_info['data_coleta'] = date.today().strftime('%d/%m/%Y')
    imovel_info['valor'] = driver.find_element(
        By.CSS_SELECTOR, value='[data-testid="price-info-value"]'
    ).text
    imovel_info['condominio'] = driver.find_element(
        By.CSS_SELECTOR, value='[id="condo-fee-price"]'
    ).text
    imovel_info['iptu'] = driver.find_element(
        By.CSS_SELECTOR, value='[id="iptu-price"]'
    ).text
    imovel_info['endereco'] = driver.find_element(
        By.CSS_SELECTOR, value='[data-testid="address-info-value"]'
    ).text

    amenities = driver.find_elements(
        By.CSS_SELECTOR, '[class="amenities-item"]'
    )

    try:
        driver.find_element(
            By.CSS_SELECTOR, '[data-cy="ldp-TextCollapse-btn"]'
        ).click()
    except:
        pass

    for amenity in amenities:
        var_amenity = amenity.get_attribute('itemprop')
        value = amenity.text  # Valor correspondente
        imovel_info[var_amenity] = value  # Armazena no dicionário

    codigo = imovel_info['codigo_zapimoveis']

    if codigo and codigo not in imoveis:
        with open('imoveis.json', 'a') as novo_imovel:
            json.dump(imovel_info, novo_imovel, ensure_ascii=False)
            novo_imovel.write('\n')  # adiciona nova linha para o próximo JSON
        imoveis.add(codigo)

    driver.close()

print('Dados exportados para imoveis.json')
driver.quit()

Acessando o link https://www.zapimoveis.com.br/imovel/venda-casa-de-condominio-3-quartos-com-piscina-serrotao-campina-grande-112m2-id-2685443938/?source=ranking%2Crp...
Acessando link 1 de 1664
Página acessada com sucesso.
Página acessada com sucesso.
Acessando o link https://www.zapimoveis.com.br/imovel/venda-apartamento-3-quartos-com-piscina-mirante-campina-grande-140m2-id-2747220909/?source=ranking%2Crp...
Acessando link 2 de 1664
Acessando o link https://www.zapimoveis.com.br/imovel/venda-apartamento-3-quartos-com-piscina-mirante-campina-grande-140m2-id-2747220909/?source=ranking%2Crp...
Acessando link 2 de 1664
Página acessada com sucesso.
Página acessada com sucesso.
Acessando o link https://www.zapimoveis.com.br/imovel/venda-casa-5-quartos-com-piscina-bela-vista-campina-grande-480m2-id-2659941819/?source=ranking%2Crp...
Acessando link 3 de 1664
Acessando o link https://www.zapimoveis.com.br/imovel/venda-casa-5-quartos-com-piscina-bela-vista-campina-grande-480m2-id-2659941819/?so